In [1]:
#loading data
import pandas as pd
Data = pd.read_csv('data.tsv', sep='\t',on_bad_lines='skip',low_memory=False)

In [2]:
Data

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


In [3]:
Data=Data.dropna()
Data = Data[Data['is_duplicate'] == 1]
Data=Data.reset_index()
Data=Data[Data['index']<100000]

In [4]:
Data

,index,id,qid1,qid2,question1,question2,is_duplicate
0,1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
1,3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
2,7,106969,147570,787.0,What is the best self help book you have read?...,What are the top self help books I should read?,1.0
3,11,233239,71243,177376.0,What will be Hillary Clinton's policy towards ...,What will be Hilary Clinton's policy towards I...,1.0
4,13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?,1.0
...,...,...,...,...,...,...,...
37021,99994,251230,62891,29406.0,What exactly is open source software?,What is open source software?,1.0
37022,99995,281278,67738,77112.0,Ow can the drive from Edmonton to Auckland be ...,How can the drive from Edmonton to Auckland be...,1.0
37023,99996,315903,211398,121844.0,Why do guys stare a lot at girls?,Why do guys like to stare at girls?,1.0
37024,99997,273368,13889,1183.0,Is backward time travel possible?,Do you think time travel is possible?,1.0


In [5]:
data=[]
for i in range(len(Data)):
    data.append(Data['question2'][i])

In [6]:
data

['How do you control your horniness?',
 'What do i do after my MBBS ?',
 'What are the top self help books I should read?',
 "What will be Hilary Clinton's policy towards India if she become President?",
 'Which is the best book for tensor calculus?',
 'What are some cool hacks for Android phones?',
 'What are some of the best motivational clips?',
 'What is the best way to reduce weight fast?',
 'How does IQ test works?',
 'Is it safe or unsafe to use Xiaomi Products?',
 'Which is the best book for cosmology?',
 'How much chances are there that NASA already knew that there is water on Mars?',
 'How can I learn more about stocks?',
 'How do I spend my long weekend in an effective way?',
 'Why do people join ISIS?',
 'Which are some of the most beautiful houses around the world?',
 "What's the most important lesson about life?",
 "What is Trump's take on Edward Snowden?",
 'How do I draw shear force and bending moment diagrams (strength of materials)?',
 'What is the easiest way to lear

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
docs_words=[]
stop_words = stopwords.words('english')
stemmer= PorterStemmer()
def preprocess(data):
    punc=['.',',','?','!','@','$','₹','/','&','-','+']
    data=data.lower()
    for j in punc:
        data=data.replace(j,'')
    words=word_tokenize(data)
    for w in words:
        if w in stop_words or len(w)==1:
            words.remove(w)
    #for i in range(len(words)):
            #words[i]=stemmer.stem(words1[i])
    return words
for i in range(len(data)):
    docs_words.append(preprocess(data[i]))

In [8]:
import numpy as np
from collections import Counter
import itertools

vocab = set(itertools.chain(*docs_words))#word library
v2i = {v: i for i, v in enumerate(vocab)}#counting number of word
i2v = {i: v for v, i in v2i.items()}#index number


idf_log= lambda x: 1 + np.log(len(docs_words) / (x+1))


tf_methods = {"log": lambda x: np.log(1+x),}

def get_tf(method="log"):
    # term frequency: how frequent a word appears in a doc
    _tf = np.zeros((len(vocab), len(docs_words)), dtype=np.float64)    # [n_vocab, n_doc]
    for i, d in enumerate(docs_words):
        counter = Counter(d)
        for v in counter.keys():
            _tf[v2i[v], i] = counter[v] / counter.most_common(1)[0][1]
 
    weighted_tf = tf_methods.get(method, None)
    if weighted_tf is None:
        raise ValueError
    return weighted_tf(_tf)


def get_idf():
    # inverse document frequency
    df = np.zeros((len(i2v), 1))
    for i in range(len(i2v)):
        d_count = 0
        for d in docs_words:
            d_count += 1 if i2v[i] in d else 0
        df[i, 0] = d_count
    return idf_log(df)


def cosine_similarity(q, _tf_idf):
    unit_q = q / np.sqrt(np.sum(np.square(q), axis=0, keepdims=True))
    unit_ds = _tf_idf / np.sqrt(np.sum(np.square(_tf_idf), axis=0, keepdims=True))
    similarity = unit_ds.T.dot(unit_q).ravel()
    return similarity


def docs_score(q, len_norm=False):
    q_words = preprocess(q)
    _idf, _tf_idf = idf, tf_idf
    counter = Counter(q_words)
    q_tf = np.zeros((len(_idf), 1), dtype=np.float64)
    for v in counter.keys():
        q_tf[v2i[v], 0] = counter[v]

    q_vec = q_tf * _idf

    q_scores = cosine_similarity(q_vec, _tf_idf)
    if len_norm:
        len_docs = [len(d) for d in docs_words]
        q_scores = q_scores / np.array(len_docs)
    return q_scores


tf = get_tf()           
idf = get_idf()       
tf_idf = tf * idf       

In [21]:
# test for TF-IDF algorithm
q = "What the best science documentaries?"
scores = docs_score(q)
d_ids = scores.argsort()[-5:][::-1]
# print("\ntop 5 docs for '{}':\n{}".format(q, [data['question2'][i] for i in d_ids]))
print("top 5 closet sentence for '{}':\n{}".format(q,[Data['question2'][i] for i in d_ids]))

top 5 closet sentence for 'What the best science documentaries?':
['What are some good science documentaries?', 'What are the best documentaries about World War 2?', 'What are your favourite documentaries? Why?', 'What are your favourite documentaries? Why?', 'What are the best World War II documentaries?']


In [22]:
Data['question1'][20]

'What are the best websites for entrepreneurs?'

In [20]:
Data['question2'][25]

'What are some good science documentaries?'